In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from selenium.webdriver.common.by import By
from selenium import webdriver
import datetime as dt
import time
import os
import glob
import numpy as np

Just a moment...Enable JavaScript and cookies to continue


In [17]:
URL = "https://coppermind.net/wiki/Category:Characters"

dr = webdriver.Chrome()
dr.implicitly_wait(2)


In [18]:
dr.get(URL)
dr.implicitly_wait(2)
dr.find_element(By.CSS_SELECTOR,'button').click()
soup = BeautifulSoup(dr.page_source, "lxml")

In [19]:
page_section = soup.findAll("div", attrs={"class":"mw-category"})[1]
all_characters = [item["href"] for item in page_section.findAll("a")]


['', 'wiki', 'Aarik']


In [ ]:

# for character in all_characters:
#     dr.get(f"https://coppermind.net{character}")
#     soup = BeautifulSoup(dr.page_source, "lxml")
#     character_name = character.split("/")[2]
#     try:
#         with open(f'characters/{character_name}.html', 'wb+') as f:
#             table = soup.find('table',attrs = {"id":"Character"})
#             table.find()
#             f.write(dr.page_source.encode("utf-8"))
#     except:
#         print(f"Error in character{character_name}")
#     time.sleep(2)

Error in characterAdoline
Error in characterAdonalsium
Error in characterArmando
Error in characterArnaud
Error in characterAudrey
Error in characterAustre
Error in characterBekenah
Error in characterChowyun_Chin
Error in characterClive
Error in characterDru
Error in characterDurk
Error in characterDylan
Error in characterFellette_Renoux
Error in characterFlavine
Error in characterFlen
Error in characterFlip
Error in characterGenedere
Error in characterHadren_Renoux
Error in characterHarl
Error in characterHarrison
Error in characterIgnacio
Error in characterIvans
Error in characterIvy
Error in characterIzumakamo
Error in characterJ.C.
Error in characterJal_Mala
Error in characterJanie
Error in characterJaxlim
Error in characterJeber
Error in characterJenny_Zhang
Error in characterJustin
Error in characterKalami
Error in characterKalyani
Error in characterKamiko
Error in characterKar
Error in characterLeroy
Error in characterLua
Error in characterMarci
Error in characterMarinda
Error i

In [14]:
print(soup)

<html class="client-js" dir="ltr" lang="en"><head>
<meta charset="utf-8"/>
<title>Category:Characters - The Coppermind - 17th Shard</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"1b2064a71e88f8e1e1bf2dde","wgCSPNonce":!1,"wgCanonicalNamespace":"Category","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":14,"wgPageName":"Category:Characters","wgTitle":"Characters","wgCurRevisionId":164735,"wgRevisionId":164735,"wgArticleId":40,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Content"],"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgRelevantPageName":"Category:Characters","wgRelevantArticleId":40,"wgIsProbablyEditable":!0,"wgRelevan

In [ ]:
dr.quit()

In [3]:
file_list = [character.split("\\")[1] for character in glob.glob("characters/*.html")]

In [ ]:
df = pd.DataFrame()

for file in file_list:
    
    character_name = file.split(".html")[0]
    temp_dict = {}
    temp_dict["Name"] = character_name

    with open(f'characters/{character_name}.html', 'rb') as f:
        soup = BeautifulSoup(f.read(), 'lxml')
        table = soup.find('table',attrs = {"id":"Character"})
        if(table.findAll("sup")):
            for sup_tag in table.findAll("sup"):
                sup_tag.decompose()
        if(table.findAll("small")):
            for sup_tag in table.findAll("small"):
                sup_tag.decompose()

        for i in table.findAll("tr", attrs={"class":"kv"}):
            category = i.find("th").text.strip()
            content = i.find("td").text.strip()
            content_list = content.split(", ")
            content_list = [i for i in content_list.strip()]
            temp_dict[category] = content_list
        
        for key in temp_dict.keys():
            if key not in df.columns:
                df[key] = pd.NA

        df.loc[len(df)] = temp_dict



Stuff to be treated:

Only Cosmere (Done)
Fix "%27" on names [like in "Wayne%27s_mother"]
Fix repeating names (think about what to do with them)
Think about what to do with titles/Aliases

Not necessarily remove: Decendants, Ancestors, Ethnicity, Nationality, Family
Maybe merge Nationality and Ethnicity


Index(['Name', 'Parents', 'Titles', 'Profession', 'Groups', 'Residence',
       'Ethnicity', 'Nationality', 'Homeworld', 'Universe', 'Introduced In',
       'Abilities', 'Species', 'Died', 'Powers', 'Siblings', 'Spouse',
       'Relatives', 'Ancestors', 'Born', 'Aliases', 'Bonded With', 'Children',
       'Descendants', 'Birthplace', 'Family', 'Era', 'Religion', 'Skills',
       'Achievements'],
      dtype='object')

In [5]:
cosmere_df = df[df['Universe'].apply(lambda x: x == ["Cosmere"])]
cosmere_df = cosmere_df.drop(columns=["Homeworld", "Family","Ethnicity","Nationality","Descendants", "Ancestors", "Universe", "Profession","Residence","Introduced In","Died","Powers","Religion","Born","Birthplace","Era", "Skills", "Achievements"])


columns_to_combine = ["Parents","Siblings","Spouse","Relatives","Children"]
cosmere_df[columns_to_combine] = cosmere_df[columns_to_combine].map(lambda x: x if isinstance(x, list) else [])
cosmere_df['All_Relatives'] = [[e for l in x for e in l] for _,x in cosmere_df.filter(items=columns_to_combine).iterrows()]
cosmere_df = cosmere_df.drop(columns=columns_to_combine)
cosmere_df.loc[~cosmere_df['All_Relatives'].astype(bool),'All_Relatives']=np.nan



cosmere_df

,Name,Titles,Groups,Abilities,Species,Aliases,Bonded With,All_Relatives
0,Aarik,[Lord General],"[Taishin, The Tower]",NaN,NaN,NaN,NaN,[Raagent]
2,Abaray,NaN,NaN,[Fused],[Singer],NaN,NaN,NaN
3,Abiajan,[Citylady],NaN,NaN,[Singer],NaN,NaN,NaN
6,Abrem,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Abrial,[Highprince of Jah Keved],NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2347,Zetah,[Zetah the Voiced],NaN,NaN,NaN,NaN,NaN,NaN
2354,Zuln,NaN,[Council of Five],NaN,[Singer],NaN,NaN,NaN
2355,Zu_(Roshar),NaN,[Knights Radiant],"[Stoneward, Shardbearer]",NaN,NaN,[Ua'pam],NaN
2356,Zu_(Sel),NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#cosmere_df[cosmere_df['Bonded With'].notnull()]
#cosmere_df[cosmere_df['Bonded With'].apply(lambda x: ("Odium" in x) if isinstance(x, list) else False)]
#cosmere_df['Abilities'].explode().unique()



['Knights Radiant (Lightweavers)', 'Unseen Court', 'Ghostbloods (formerly)']

In [6]:

def find_similar_items(df, name1, name2):
    # Initialize an empty dictionary to store similarities
    similarities = {}
    
    # Get the rows corresponding to each name
    row1 = df[df['Name'] == name1].iloc[0]
    row2 = df[df['Name'] == name2].iloc[0]
    # Loop over each column in the dataframe
    for column in df.columns:
        # Check if the column contains lists (e.g., if it's a string representation of a list or an actual list)
        if isinstance(row1[column], list) and isinstance(row2[column], list):

            # Find the intersection of both lists
            common_items = list(set(row1[column]).intersection(set(row2[column])))
            if common_items:
                similarities[column] = common_items
    
    return similarities

In [7]:
similarities = 1
name1 = "Kaladin"


while(similarities):
    name2=input()
    similarities = find_similar_items(cosmere_df, name1, name2)
    print(similarities)

    name1 = name2




{'Groups': ['Knights Radiant '], 'Abilities': ['Shardbearer']}
{}


In [ ]:
#cosmere_df[cosmere_df['Name'] == "Shallan_Davar"].iloc[0]["Abilities"]
#cosmere_df[cosmere_df['Name'] == "Hoid"].iloc[0]["Abilities"]

['Lightweaver', 'Shardbearer']